<img src="https://raw.githubusercontent.com/FUlyankin/r_probability/master/end_seminars/sem_6/student_teacher_dist.png" height="300" width="600"> 

# <center> R для тервера и матстата. <br>  <br> 6.2 Проверка гипотез. Бустрап и правдоподобие. </center>

Данный ноутбук является конспектом по курсу «R для теории вероятностей и математической статистики» (РАНХиГС, 2017-2018). Автор ноутбука [вот этот парень по имени Филипп.](https://vk.com/ppilif) Если у вас для него есть деньги, слава или женщины, он от этого всего не откажется. Ноутбук распространяется на условиях лицензии [Creative Commons Attribution-Share Alike 4.0.](https://creativecommons.org/licenses/by-sa/4.0/) При использовании обязательно упоминание автора курса и аффилиации. При наличии технической возможности необходимо также указать активную гиперссылку на [страницу курса.](https://fulyankin.github.io/R_probability/) На ней можно найти другие материалы. Фрагменты кода, включенные в этот notebook, публикуются как [общественное достояние.](https://creativecommons.org/publicdomain/zero/1.0/)

---------

В прошлой тетрадке мы обсудили небольшое количество критериев для простых известных распределений. В мире осталось ещё куча неизведанного. И с этим неизведанным хочется расправляться одной левой. В этой тетрадке пойдёт речь о борьбе с неизвестностью. Будет затронуто три сюжета: 

In [3]:
df = read.csv('/Users/fulyankin/Yandex.Disk.localized/R/R_prob_data/flat.csv', sep='\t') 
head(df)

n,price,totsp,livesp,kitsp,dist,metrdist,walk,brick,floor,code
1,81,58,40,6,12.5,7,1,1,1,3
2,75,44,28,6,13.5,7,1,0,1,6
3,128,70,42,6,14.5,3,1,1,1,3
4,95,61,37,6,13.5,7,1,0,1,1
5,330,104,60,11,10.5,7,0,1,1,3
6,137,76,50,9,11.0,7,1,1,1,8


In [15]:
x = log(df$price)

lnL <- function(param,data) {
  n <- length(data)
  result <- -n/2*log(2*pi)-n/2*log(param[2]^2)-sum((data-param[1])^2)/(2*param[2]^2)
  return(result)
}

In [12]:
mean(x)
sd(x)

[1] 4.791375

[1] 0.3152367

In [31]:
library('maxLik') 
res <- maxLik(lnL, start=c(0.1,0.1),data=x)
summary(res)

--------------------------------------------
Maximum Likelihood estimation
Newton-Raphson maximisation, 16 iterations
Return code 1: gradient close to zero
Log-Likelihood: -539.0942 
2  free parameters
Estimates:
     Estimate Std. error t value Pr(> t)    
[1,] 4.791375   0.006978  686.66  <2e-16 ***
[2,] 0.315159   0.004934   63.88  <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
--------------------------------------------

In [32]:
par = res$estimate
lnL(par, x)

[1] -539.0942

In [33]:
lnL_res <- function(sigma,data) lnL(c(5,sigma),x)
res_restr <- maxLik(lnL_res, start=0.1,data=x)
summary(res_restr)
par_restr <- c(5, res_restr$estimate)

--------------------------------------------
Maximum Likelihood estimation
Newton-Raphson maximisation, 9 iterations
Return code 2: successive function values within tolerance limit
Log-Likelihood: -909.7547 
1  free parameters
Estimates:
     Estimate Std. error t value Pr(> t)    
[1,] 0.377955   0.005917   63.87  <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
--------------------------------------------

In [34]:
2*(lnL(par, x) - lnL(par_restr, x))

[1] 741.321

In [9]:
log_lik(c(1,2),x)

[1] -6979.486

В [табличке](https://yadi.sk/i/hDxMJ0fw3VSDmu) лежит информация о стоимости квартир в Москве и о основных параметрах этих квартир. Пусть цена на квартиры имеет логарифмически нормальное распределение. Оцените параметры $\mu$ и $\sigma$ методом максимального правдоподобия. Постройте для оценок доверительные интервалы. 